In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.image as img
import PIL.Image

In [ ]:
train = np.load('../dog-breed-identification/train.zip')

In [2]:
import glob
import random
import base64
import pandas as pd

from PIL import Image
from io import BytesIO
from IPython.display import HTML

def get_thumbnail(path): 
    image = img.imread(path)
    return image

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'

In [ ]:
dogs = pd.read_csv('../dog-breed-identification/labels.csv')
#dogs = dogs.sample(50)
dogs['file'] = dogs.id.map(lambda id: f'/home/diegotakei/workspace/dog/dog-breed-identification/train/{id}.jpg')
dogs['image'] = dogs.file.map(lambda f: get_thumbnail(f))
dogs.head()

In [4]:
dogs = pd.read_csv('../dogs_data/labels.csv')
dogs['file'] = dogs.id.map(lambda id: f'../dogs_data/resized/{id}.jpg')

In [8]:
imgs = []
for f in dogs['file'].values:
    imgs.append(get_thumbnail(f))
imgs = np.array(imgs)

imgs.shape

(10222, 256, 256, 3)

In [9]:
imgs

array([[[[112,  58,  34],
         [152,  98,  72],
         [154, 101,  69],
         ...,
         [198, 162,  68],
         [199, 159,  63],
         [218, 176,  78]],

        [[134,  81,  63],
         [186, 134, 110],
         [149,  99,  62],
         ...,
         [235, 199, 103],
         [201, 161,  65],
         [202, 162,  64]],

        [[110,  63,  43],
         [141,  96,  67],
         [139,  97,  49],
         ...,
         [246, 210, 113],
         [214, 176,  77],
         [221, 183,  82]],

        ...,

        [[200, 120,  71],
         [187, 109,  60],
         [195, 122,  69],
         ...,
         [221, 159, 122],
         [227, 170, 127],
         [216, 162, 115]],

        [[195, 118,  64],
         [183, 108,  53],
         [196, 125,  69],
         ...,
         [197, 132,  92],
         [211, 153, 105],
         [218, 162, 111]],

        [[191, 116,  59],
         [178, 106,  48],
         [195, 126,  67],
         ...,
         [165, 100,  58],
        

In [ ]:
dogs['Indexes'] = dogs["breed"].str.startswith('appenzeller')
    
result = dogs[(dogs.Indexes == True)]
result.tail()

pd.set_option('display.max_colwidth', -1)
HTML(result[['breed', 'image']].to_html(formatters={'image': image_formatter}, escape=False))
#pd.reset_option('all')

In [ ]:
train_images = imgs

train_labels = dogs['breed']

In [ ]:
def create_network(features, labels, mode):
    i = tf.reshape(features['x'], [-1, 256, 256, 3])
    
    # receives [batch_size, 256, 256, 3]
    # returns [batch_size, 256, 256, 32]
    convolution1 = tf.layers.conv2d(inputs = i, filters = 32, kernel_size = [5,5], activation = tf.nn.relu,
                                 padding = 'same')
    
    # receives [batch_size, 256, 256, 3]
    # returns [batch_size, 128, 128, 32]
    pooling1 = tf.layers.max_pooling2d(inputs = convolution1, pool_size = [2,2], strides = 2)
    
    # receives [batch_size, 128, 128, 32]
    # returns [batch_size, 128, 128, 64]
    convolution2 = tf.layers.conv2d(inputs = pooling1, filters = 64, kernel_size = [5,5], activation = tf.nn.relu,
                                  padding = 'same')
    
    # receives [batch_size, 128, 128, 64]
    # returns [batch_size, 64, 64, 64]
    pooling2 = tf.layers.max_pooling2d(inputs = convolution2, pool_size = [2,2], strides = 2)
    
    print("TUDO CERTO")
    return
    
    # receives [batch_size, 7, 7, 64]
    # returns [batch_size, 3136]
    flattening = tf.reshape(pooling2, [-1, 7 * 7 * 64])
    
    # 3136 inputs -> 1024 neurons on hidden layer -> 10 outputs
    # receives [batch_size, 3136]
    # returns [batch_size, 1024]
    dense = tf.layers.dense(inputs = flattening, units = 1024, activation = tf.nn.relu)
    
    dense2 = tf.layers.dense(inputs = dense, units = 1024, activation = tf.nn.relu)
    
    # dropout
    dropout =  tf.layers.dropout(inputs = dense2, rate = 0.2, training = mode == tf.estimator.ModeKeys.TRAIN)
    
    # receives [batch_size, 1024]
    # returns [batch_size, 10]
    output = tf.layers.dense(inputs = dropout, units = 10)
    
    predictions = tf.argmax(output, axis = 1)
    
    if(mode == tf.estimator.ModeKeys.PREDICT):
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)  
    
    losses = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = output)
    
    if(mode == tf.estimator.ModeKeys.TRAIN):
        optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
        train = optimizer.minimize(losses, global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = losses, train_op = train)
    
    if(mode == tf.estimator.ModeKeys.EVAL):
        eval_metrics_ops = {'accuracy': tf.metrics.accuracy(labels = labels, predictions = predictions)}
        return tf.estimator.EstimatorSpec(mode = mode, loss = losses, eval_metric_ops = eval_metrics_ops) 

classifier = tf.estimator.Estimator(model_fn = create_network)

train_function = tf.estimator.inputs.numpy_input_fn(x = {'x': train_images}, y = train_labels, 
                                                        batch_size= 10000, num_epochs= None, shuffle= True)
classifier.train(input_fn = train_function, steps = 2000)